In [ ]:
%load_ext watermark


In [ ]:
import itertools as it
import os
import typing

from downstream import dstream
from IPython.display import display, HTML
import matplotlib as mpl
from matplotlib import ticker as mpl_ticker
import more_itertools as mit
import numpy as np
import polars as pl
import seaborn as sns
from slugify import slugify
from teeplot import teeplot as tp

import pylib  # noqa: F401


In [ ]:
%watermark -diwmuv -iv


In [ ]:
tp.save[".pgf"] = False
teeplot_subdir = os.environ.get(
    "NOTEBOOK_NAME", "2025-07-06-qos-dstream-tilted"
)
teeplot_subdir


In [ ]:
num_items = 10_000


## Define


In [ ]:
def calc_max_qos_from_segment_lengths(segment_lengths: list[int]) -> float:
    segment_total = sum(segment_lengths)
    return max(
        (segment_length - 1) / (
            segment_total - cumulative - segment_length + 1 or 1
        )
        for cumulative, segment_length in zip(
            it.accumulate([0, *segment_lengths]),
            [*segment_lengths, 1],
        )
    )


In [ ]:
def calc_mean_qos_from_segment_lengths(segment_lengths: list[int]) -> float:
    segment_total = sum(segment_lengths)
    segment_fenceposts = it.accumulate([0, *segment_lengths])
    values = []
    for begin, end in it.pairwise(segment_fenceposts):
        for rank in range(begin, end):
            depth = segment_total - rank
            assert depth
            values.append((end - begin - 1) / depth)

    values.append(0)  # fencepost

    assert len(values) ==  segment_total + 1
    assert max(values) == calc_max_qos_from_segment_lengths(segment_lengths)
    return np.mean(values)


In [ ]:
def calc_max_gaps_naive_doubling_tilted(
    buffer_size: int,
    num_ingests: int,
    calc_qos: typing.Callable,
) -> list[int]:
    segment_lengths = []
    max_gaps = [0]
    for i in range(num_ingests):

        if (len(segment_lengths) == buffer_size):
            segment_lengths = [
                a + b
                for a, b in mit.batched(segment_lengths, 2)
            ]
        segment_lengths.append(1)

        assert sum(segment_lengths) == i + 1
        max_gaps.append(calc_qos(segment_lengths))

    return max_gaps[:-1]


In [ ]:
def calc_max_gaps_dstream(
    buffer_size: int,
    num_items: int,
    calc_qos: typing.Callable,
) -> list[int]:
    inventories = dstream.tilted_algo.lookup_ingest_times_batched(
        buffer_size,
        np.arange(buffer_size, num_items),
    ).astype(np.int64)
    sorted_inventories = np.sort(
        inventories,
        axis=1,
    )
    gaps = np.diff(sorted_inventories, axis=1)
    return [0] * buffer_size + [
        calc_qos(gap_lengths)
        for gap_lengths in gaps
    ]


In [ ]:
def calc_max_gaps_zhao_tilted(
    buffer_size: int,
    num_items: int,
    calc_qos: typing.Callable,
) -> list[int]:
    segment_lengths = []
    max_gaps = [0]
    for k in range(num_items):

        segment_lengths.append(1)
        for i, j in it.pairwise(reversed(range(len(segment_lengths) - 1))):
            if segment_lengths[i] == segment_lengths[j]:
                segment_lengths[i] += segment_lengths[j]
                del segment_lengths[j]
                break

        assert sum(segment_lengths) == k + 1
        max_gaps.append(calc_qos(segment_lengths))

    return max_gaps[:-1]


In [ ]:
def calc_max_gaps_zhao_tilted_full(
    buffer_size: int,
    num_items: int,
    calc_qos: typing.Callable,
) -> list[int]:
    bucket_sizes = [buffer_size, 0]
    buffer = [*range(buffer_size)]
    max_gaps = [0]


    S = buffer_size
    w = bucket_sizes
    for k in range(num_items):
        if k < S:
            assert buffer[k] == k
        else:
            bucket_sizes[0] += 1
            i = S
            j = 0
            while w[j] <= w[j + 1]:
                i -= w[j]
                j += 1
                if j == len(w) - 1:
                    w.append(0)

            assert 0 <= i <= S
            w[j] -= 2
            w[j + 1] += 1
            for n in range(i - w[j], S - 1):
                assert 0 < n < S - 1
                buffer[n] = buffer[n + 1]
            buffer[S - 1] = k

        assert buffer == sorted(buffer)
        assert sum(bucket_sizes) == buffer_size

        segment_lengths = [
            b - a for a, b in mit.pairwise(sorted(buffer))
        ]
        assert (
            sum(segment_lengths)
            == buffer_size + max(k - buffer_size + 1, 0) - 1
        )
        max_gaps.append(
            calc_qos(segment_lengths),
        )

    return max_gaps[:-1]


## Example Plot


In [ ]:
def make_df(buffer_size: int) -> pl.DataFrame:
    return pl.concat(
        [
            pl.DataFrame(
                {
                    "Algorithm": "naive doubling",
                    "Gap Size Cost": calc_max_gaps_naive_doubling_tilted(
                        buffer_size,
                        num_items,
                        calc_max_qos_from_segment_lengths,
                    ),
                    "Gap Size Cost Mean": calc_max_gaps_naive_doubling_tilted(
                        buffer_size,
                        num_items,
                        calc_mean_qos_from_segment_lengths,
                    ),
                    "Num Items Ingested": range(num_items),
                },
                strict=False,
            ),
            pl.DataFrame(
                {
                    "Algorithm": "extended ringbuf",
                    "Gap Size Cost": calc_max_gaps_dstream(
                        buffer_size,
                        num_items,
                        calc_max_qos_from_segment_lengths,
                    ),
                    "Gap Size Cost Mean": calc_max_gaps_dstream(
                        buffer_size,
                        num_items,
                        calc_mean_qos_from_segment_lengths,
                    ),
                    "Num Items Ingested": range(num_items),
                },
                strict=False,
            ),
            pl.DataFrame(
                {
                    "Algorithm": "pyramidal bucket",
                    "Gap Size Cost": calc_max_gaps_zhao_tilted(
                        buffer_size,
                        num_items,
                        calc_max_qos_from_segment_lengths,
                    ),
                    "Gap Size Cost Mean": calc_max_gaps_zhao_tilted(
                        buffer_size,
                        num_items,
                        calc_mean_qos_from_segment_lengths,
                    ),
                    "Num Items Ingested": range(num_items),
                },
                strict=False,
            ),
            pl.DataFrame(
                {
                    "Algorithm": "saturating bucket",
                    "Gap Size Cost": calc_max_gaps_zhao_tilted_full(
                        buffer_size,
                        num_items,
                        calc_max_qos_from_segment_lengths,
                    ),
                    "Gap Size Cost Mean": calc_max_gaps_zhao_tilted_full(
                        buffer_size,
                        num_items,
                        calc_mean_qos_from_segment_lengths,
                    ),
                    "Num Items Ingested": range(num_items),
                },
                strict=False,
            ),
        ]
    )


In [ ]:
hue_order = [
    "extended ringbuf",
    "naive doubling",
    "saturating bucket",
    "pyramidal bucket",
]

palette = {
    "extended ringbuf": "#9C4897",
    "naive doubling": "#828282",
    "saturating bucket": "#63a28b",
    "pyramidal bucket": "#baf23d",
}

markers = {
    "extended ringbuf": "X",
    "naive doubling": "d",
    "saturating bucket": "d",
    "pyramidal bucket": ".",
}

dashes = [(5, 2), (1, 0), (1, 0), (1, 1)]


In [ ]:
for buffer_size in [64, 256, 1024]:
    df = make_df(buffer_size)
    for what, excl, yscale, legend in it.product(
        ["Gap Size Cost", "Gap Size Cost Mean"],
        [["naive doubling", "pyramidal bucket"], ["naive doubling"], []],
        ["symlog", "linear"],
        [True, False]
    ):
        display(HTML(f"<h2>{what}</h2>"))
        ho = [x for x in hue_order if x not in excl]

        for rc in [{}, {"font.family": "serif"}]:
            with mpl.rc_context(rc=rc):
                with tp.teed(
                    sns.relplot,
                    df.filter(
                        ~pl.col("Algorithm").is_in(excl),
                    ),
                    x="Num Items Ingested",
                    y=what,
                    hue="Algorithm",
                    hue_order=ho,
                    style_order=ho,
                    style="Algorithm",
                    markers=markers,
                    aspect=2.2,
                    dashes=[(1, 0)] * 4,
                    kind="line",
                    markersize=0,
                    height=1.35,
                    legend=legend,
                    palette=palette,
                    teeplot_outattrs={
                        "buffer_size": buffer_size,
                        "excl": "-".join(map(slugify, excl)),
                        "yscale": yscale,
                        **({"legend": legend} if not legend else {}),
                        **rc,
                    },
                    teeplot_subdir=teeplot_subdir,
                ) as g:
                    if legend:
                        sns.move_legend(
                            g,
                            "lower center",
                            bbox_to_anchor=(0.38, 1),
                            ncol=2,
                            title=None,
                            frameon=False,
                        )
                    sep = " " if legend else "\n"
                    g.set(ylabel=f"Gap Size{sep}Ratio", xlabel="Time ($T$)")
                    g.set(yscale=yscale)
                    sns.scatterplot(
                        data=df.filter(
                            ~pl.col("Algorithm").is_in(excl),
                        ).filter(
                            pl.col("Num Items Ingested")
                            == pl.col("Num Items Ingested").max(),
                        ).with_columns(
                            pl.col("Num Items Ingested") * pl.when(
                                pl.col("Algorithm") == "extended ringbuf"
                            ).then(1.08).when(
                                pl.col("Algorithm") == "saturating bucket"
                            ).then(1.03).otherwise(1.05),
                        ),
                        x="Num Items Ingested",
                        y=what,
                        hue="Algorithm",
                        hue_order=ho,
                        style="Algorithm",
                        style_order=ho,
                        markers=markers,
                        s=100,
                        palette=palette,
                        legend=False,
                        ax=g.axes.flat[0],
                        clip_on=False,
                    )
                    g.axes.flat[0].yaxis.set_minor_locator(
                        {
                            "symlog": mpl_ticker.LogLocator(
                                numticks=999, subs="auto"
                            ),
                            "linear": mpl_ticker.AutoMinorLocator(5),
                        }[yscale]
                    )
                    if legend:
                        for handle in g._legend.legend_handles:
                            handle.set_markersize(8)

                if not excl:
                    dfp = df.filter(
                            ~pl.col("Algorithm").is_in(excl),
                        ).pivot(
                        index="Num Items Ingested",
                        on="Algorithm",
                        values=what,
                    ).to_pandas()
                    dfp["ratio"] = (
                        dfp["saturating bucket"] / dfp["extended ringbuf"]
                    )
                    inf, nan = np.inf, np.nan
                    print(
                        f"{dfp['ratio'].min()=:.2f}",
                        f"{dfp['ratio'].replace(inf, nan).dropna().mean()=:.2f}",
                        sep="\n",
                    )
                    with tp.teed(
                        sns.relplot,
                        dfp,
                        x="Num Items Ingested",
                        y="ratio",
                        aspect=2.0,
                        kind="line",
                        height=1.8,
                        palette="Set2",
                        teeplot_outattrs={
                            "buffer_size": buffer_size,
                            "what": slugify(what),
                            "yscale": yscale,
                            **rc,
                        },
                        teeplot_subdir=teeplot_subdir,
                    ) as g:
                        g.set(ylim=(0, 1), yscale=yscale)
                        g.set(
                            ylabel="extended\nringbuf \n/ saturating\nbucket",
                        )
                        g.axes.flat[0].yaxis.set_minor_locator(
                            mpl_ticker.AutoMinorLocator(5),
                        )
